In [ ]:
%matplotlib inline

# Classification with pre-trained model on CIFAR10

CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html> is a
dataset of tiny (32x32) images with labels, collected by Alex Krizhevsky,
Vinod Nair, and Geoffrey Hinton. It is widely used as benchmark in
computer vision research.

In this tutorial, we will demonstrate how to load a pre-trained model from `gluoncv-model-zoo`
and classify images from the Internet or your local disk.

In [ ]:
from IPython.display import Image
Image(url= "https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/datasets/cifar10.png")

In [ ]:
import matplotlib.pyplot as plt

from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv import utils
from gluoncv.model_zoo import get_model

Then, we download and show the example image:



In [ ]:
url = 'https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/classification/plane-draw.jpeg'
im_fname = utils.download(url)

img = image.imread(im_fname)

plt.imshow(img.asnumpy())
plt.show()

In case you don't recognize it, the image is a poorly-drawn airplane :)

Now we define transformations for the image.



In [ ]:
transform_fn = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

This transformation function does three things:
resize and crop the image to 32x32 in size,
transpose it to `num_channels*height*width`,
and normalize with mean and standard deviation calculated across all CIFAR10 images.

What does the transformed image look like?



In [ ]:
img = transform_fn(img)
plt.imshow(nd.transpose(img, (1,2,0)).asnumpy())
plt.show()

Can't recognize anything? *Don't panic!* Neither do I.
The transformation makes it more "model-friendly", instead of "human-friendly".

Next, we load a pre-trained model.



In [ ]:
net = get_model('cifar_resnet110_v1', classes=10, pretrained=True)

Finally, we prepare the image and feed it to the model



In [ ]:
pred = net(img.expand_dims(axis=0))

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
ind = nd.argmax(pred, axis=1).astype('int')
print('The input picture is classified as [%s], with probability %.3f.'%
      (class_names[ind.asscalar()], nd.softmax(pred)[0][ind].asscalar()))